## Streamlining carpooling in ride-sharing platforms using Graph Theory
In my recent work, I've harnessed the power of graph theory to streamline carpooling in ride-sharing platforms. Using Python, I created a directed graph to simulate locations and their interconnections, with weights representing distances. The key here is leveraging Dijkstra's algorithm to find the shortest paths for carpooling routes. This method not only makes ride-sharing more efficient but also reduces travel time and costs. My next step is to incorporate real-time traffic data, further optimizing these routes and enhancing the ride-sharing experience for both drivers and riders.

### Import libraries

In [1]:
import dash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_bootstrap_components as dbc
import networkx as nx
import random

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until


### Constructing a Directed Graph for Location-Based Analysis
This code snippet creates a directed graph representing a network of 20 locations, with nodes for each location and edges indicating the distance between them. Each edge has a randomly assigned weight, representing the distance, making the graph useful for location-based network analysis.

In [2]:
# Create nodes (representing locations)
locations = [f"L{i}" for i in range(20)]

# Create a directed graph
G = nx.DiGraph()

# Nodes are added to the Graph
G.add_nodes_from(locations)

# Add edges with weights (representing distances between locations)
for location1 in locations:
    for location2 in locations:
        if location1 != location2:
            distance = random.randint(1, 100)
            G.add_edge(location1, location2, weight=distance)
            G.add_edge(location2, location1, weight=distance)  # symmetric weight

### Optimizing Carpool Routes with Dijkstra's Algorithm
The code selects a base location and three unique destinations, then uses Dijkstra's algorithm to find the shortest paths sequentially from the base to each destination. This approach helps in planning efficient carpool routes by connecting the chosen locations through the shortest possible paths.

In [3]:
# Choose one base location and three destinations
base_location = random.choice(locations)
destinations = random.sample([loc for loc in locations if loc != base_location], 3)  # Ensure unique destinations

# Find shortest path from base to each destination and combine them
path_to_dest1 = nx.dijkstra_path(G, source=base_location, target=destinations[0])
path_to_dest2 = nx.dijkstra_path(G, source=destinations[0], target=destinations[1])
path_to_dest3 = nx.dijkstra_path(G, source=destinations[1], target=destinations[2])

# Print for debugging purposes
print("Path from base to destination 1:", path_to_dest1)
print("Path from destination 1 to destination 2:", path_to_dest2)
print("Path from destination 2 to destination 3:", path_to_dest3)

Path from base to destination 1: ['L6', 'L7', 'L18']
Path from destination 1 to destination 2: ['L18', 'L7', 'L16', 'L0', 'L8']
Path from destination 2 to destination 3: ['L8', 'L19']


### Optimizing Carpool Routes Using Network Graph Analysis
The code combines multiple shortest paths to create a single carpool route and converts this route along with other graph data into a format compatible with Dash Cytoscape for visualization. It defines a custom stylesheet for the visual elements, highlighting different nodes and paths, and calculates distances along the route. This approach efficiently illustrates the carpooling paths in a network, optimizing routes based on distances between locations

In [4]:
# Combine all paths
full_path = path_to_dest1 + path_to_dest2[1:] + path_to_dest3[1:]

# Convert NetworkX Graph to cytoscape data format
cytoscape_format_data = nx.readwrite.json_graph.cytoscape_data(G)

# Convert data to Dash Cytoscape elements
elements = cytoscape_format_data['elements']['nodes'] + cytoscape_format_data['elements']['edges']

# Define stylesheet
stylesheet=[
    {
        'selector': 'node',
        'style': {
            'content': 'data(id)',
            'color': 'white',
            'text-valign': 'center',
            'text-halign': 'center',
            'background-color': 'blue',
        }
    },
    {
        'selector': 'edge',
        'style': {
            'curve-style': 'bezier',
            'label': 'data(weight)',
            'target-arrow-shape': 'triangle',
            'line-color': 'black',
            'line-style': 'dotted',
            'width': 1
        }
    },
    # Change the color of the start and end nodes
    {
        'selector': f'node[id = "{base_location}"]',
        'style': {
            'background-color': 'red',
        }
    },
    {
        'selector': f'node[id = "{destinations[0]}"]',
        'style': {
            'background-color': 'green',
        }
    },
    {
        'selector': f'node[id = "{destinations[1]}"]',
        'style': {
            'background-color': 'magenta',
        }
    },
    {
        'selector': f'node[id = "{destinations[2]}"]',
        'style': {
            'background-color': 'orange',
        }
    },
]

# Highlight the full path
for i in range(len(full_path) - 1):
    stylesheet.append({
        'selector': f'edge[source = "{full_path[i]}"][target = "{full_path[i + 1]}"]',
        'style': {
            'line-color': 'red',
            'line-style': 'solid',
            'width': 2
        }
    })

# Calculate distances
dist_to_dest1 = nx.dijkstra_path_length(G, source=base_location, target=destinations[0])
dist_to_dest2 = nx.dijkstra_path_length(G, source=destinations[0], target=destinations[1])
dist_to_dest3 = nx.dijkstra_path_length(G, source=destinations[1], target=destinations[2])

# Construct the path string
path_string = (f"{base_location} --- ({dist_to_dest1}) --> {destinations[0]} "
               f"--- ({dist_to_dest2}) --> {destinations[1]} "
               f"--- ({dist_to_dest3}) --> {destinations[2]}")

print(path_string)

L6 --- (16) --> L18 --- (26) --> L8 --- (10) --> L19


### Building an Interactive Dash App for Visualizing Optimal Routes
This code snippet sets up a Dash web application to visualize the shortest path in a network. It uses Bootstrap for styling and integrates Cytoscape for graph visualization, displaying the calculated shortest path and the network's structure interactively.

In [5]:
# Dash App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Row(
        dbc.Col(html.H3("Shortest Path: " + path_string), width=12)
    ),
    dbc.Row(
        dbc.Col(cyto.Cytoscape(
            id='cytoscape-elements',
            layout={'name': 'random'},
            style={'width': '100%', 'height': '900px'},
            elements=elements,
            stylesheet=stylesheet
        ), width=12)
    ),
], fluid=True)

if __name__ == '__main__':
    app.run_server(debug=True)
